In [1]:
import sys
sys.path.append('../../../')
from utils.packages import *
from utils.ml_fairness import *
from utils.standard_data import *
dir = 'res/german1/'
Path(dir).mkdir(parents=True, exist_ok=True)

d_fields = ['Stage', 'CVR', 'CVD', 'V_SPD', 'V_EOD', 'V_AOD', 'V_ERD', 'Acc', 'F1','SPD', 'EOD', 'AOD', 'ERD']
diff_file = dir + 'diff' + '.csv'
if(not os.path.isfile(diff_file)):
    with open(diff_file, 'a') as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(d_fields)
    
f_count = len([name for name in os.listdir(dir) if os.path.isfile(os.path.join(dir, name)) and not name.startswith('.')])
# fields = ['Acc', 'F1', 'DI','SPD', 'EOD', 'AOD', 'ERD', 'CNT', 'TI']
fields = ['Acc', 'F1', 'SPD', 'EOD', 'AOD', 'ERD']

filename = dir + 'glob-' + str(f_count) + '.csv'
with open(filename, 'a') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(fields)


In [2]:
filepath = '../../../data/german/german.data'
column_names = ['status', 'month', 'credit_history',
            'purpose', 'credit_amount', 'savings', 'employment',
            'investment_as_income_percentage', 'personal_status',
            'other_debtors', 'residence_since', 'property', 'age',
            'installment_plans', 'housing', 'number_of_credits',
            'skill_level', 'people_liable_for', 'telephone',
            'foreign_worker', 'credit']
na_values=[]
df = pd.read_csv(filepath, sep=' ', header=None, names=column_names,na_values=na_values)
# df['age'] = df['age'].apply(lambda x: np.float(x >= 26))
df = german_custom_preprocessing(df)
feat_to_drop = ['personal_status']
df = df.drop(feat_to_drop, axis=1)

cat_feat = ['status', 'credit_history', 'purpose', 'savings', 'employment', 'other_debtors', 'property', 'installment_plans', 'housing', 'skill_level', 'telephone', 'foreign_worker']
df = pd.get_dummies(df, columns=cat_feat, prefix_sep='=')


In [9]:
for i in range(5):
    
    seed = randrange(100)
    y2_train, y2_test = train_test_split(df, test_size = 0.3, random_state = seed) # stratify=df['race']
    y1_train, y1_test = train_test_split(df, test_size = 0.3, random_state = seed) # 

    pro_att_name = ['sex'] # ['sex', 'age']
    priv_class = [1]
    reamining_cat_feat = []

    y2_data_orig_train, y2_X_train, y2_y_train = load_german_data(y2_train, pro_att_name, priv_class, reamining_cat_feat)
    y2_data_orig_test, y2_X_test, y2_y_test = load_german_data(y2_test, pro_att_name, priv_class, reamining_cat_feat)

    y1_data_orig_train, y1_X_train, y1_y_train = load_german_data(y1_train, pro_att_name, priv_class, reamining_cat_feat)
    y1_data_orig_test, y1_X_test, y1_y_test = load_german_data(y1_test, pro_att_name, priv_class, reamining_cat_feat)
    
    y1_test_df = y1_data_orig_test.copy()
    ## Feature engineering
    pca = PCA(n_components=3)

    trained = pca.fit(y2_X_train)
    y2_X_train = trained.transform(y2_X_train)
    y2_X_test = trained.transform(y2_X_test)
    y2_data_orig_test.features = y2_X_test
    
    trained_f = pca.fit(y1_X_train)
    y1_X_train = trained_f.transform(y1_X_train)
    y1_X_test = trained_f.transform(y1_X_test)
    y1_data_orig_test.features = y1_X_test

    from sklearn.feature_selection import SelectKBest
    skb = SelectKBest(k=2)

    trained_f = skb.fit(y2_X_train, y2_y_train)
    y2_X_train = trained_f.transform(y2_X_train)
    y2_X_test = trained_f.transform(y2_X_test)
    y2_data_orig_test.features = y2_X_test
    
#     trained_f = skb.fit(y1_X_train, y1_y_train)
#     y1_X_train = trained_f.transform(y1_X_train)
#     y1_X_test = trained_f.transform(y1_X_test)
#     y1_data_orig_test.features = y1_X_test


    #Seting the Hyper Parameters
    param_grid = {"max_depth": [3,5, 7, 10,None],
                  "n_estimators":[3,5,10,25,50,150],
                  "max_features": [2,4,7,15,20]}

    y2_model = GridSearchCV(RandomForestClassifier(), param_grid={}, cv=5, scoring='recall', verbose=0)
    y2_mdl = y2_model.fit(y2_X_train, y2_y_train)

    y1_model = GridSearchCV(RandomForestClassifier(), param_grid={}, cv=5, scoring='recall', verbose=0)
    y1_mdl = y1_model.fit(y1_X_train, y1_y_train)
    

    # plot_model_performance(y2_mdl, y2_X_test, y2_y_test)
    y1_pred, y1_fair = get_fair_metrics_and_plot(filename, y1_data_orig_test, y1_mdl)
    y2_pred, y2_fair = get_fair_metrics_and_plot(filename, y2_data_orig_test, y2_mdl)



    y1_fair = y1_fair.drop(['DI', 'CNT', 'TI'], axis=1)
    y2_fair = y2_fair.drop(['DI', 'CNT', 'TI'], axis=1)
    CVR, CVD, AVR_EOD, AVD_EOD, AVR_SPD, AVD_SPD, AVD_AOD, AV_ERD = compute_new_metrics(y1_test_df, y1_pred, y2_pred)
    row_y1 = y1_fair.iloc[[0]].values[0].tolist()
    row_y2 = y2_fair.iloc[[0]].values[0].tolist()
    diff = []

    with open(filename, 'a') as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(row_y1)
        csvwriter.writerow(row_y2) 
    
    diff.append(CVR)
    diff.append(CVD)
    diff.append(AVD_SPD)
    diff.append(AVD_EOD)
    diff.append(AVD_AOD)
    diff.append(AV_ERD)

    for i in range(len(row_y2)):
        if(i < 2):
            change = row_y2[i] - row_y1[i]
        else:
            sign = ''
            if(row_y2[i] >= 0 and row_y1[i] >= 0):
                sign = '(+)'
                d = abs(row_y2[i]) - abs(row_y1[i])
            if(row_y2[i] < 0 and row_y1[i] < 0):
                sign = '(-)'
                d = abs(row_y2[i]) - abs(row_y1[i])
            if(row_y2[i] < 0 and row_y1[i] >= 0):
                sign = '(+-)'
                d = row_y2[i] - row_y1[i]
            if(row_y2[i] >=0 and row_y1[i] < 0):
                sign = '(-+)'
                d = row_y2[i] - row_y1[i]
            d = round(d, 3)
            change = sign + ' ' + str(d)

        diff.append(change)

    cols = ['CVR', 'CVD ', 'AV_SPD', 'AV_EOD', 'AV_AOD', 'AV_ERD', 'Acc', 'F1','SPD', 'EOD', 'AOD', 'ERD']
    # metrics = pd.DataFrame(data=obj_fairness, index=['y1'], columns=cols)
    diff_df = pd.DataFrame(data=[diff], columns  = cols, index = ['Diff']).round(3)
    stage = 'SelectKBest'
    model_name = 'german1'
    diff = diff_df.iloc[0].values.tolist()
    diff.insert(0, stage)
    diff.insert(0, model_name)
    with open(diff_file, 'a') as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(diff)    
        diff_df

Unprinv: sex 0.0
Unprinv: sex 0.0
Unprinv: sex 0.0
Unprinv: sex 0.0
Unprinv: sex 0.0
